In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
import pandas as pd
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import datetime
import time
from dateutil.parser import parse
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle
import xlab
import operator
from sklearn import preprocessing

def getDateType():
    date=pd.read_table('data/da0030/LV35_T_TDW_BOPR_OPR_T80_WKD_PARM_INF_20190130_000AH000.DAT',engine='python',sep = '|',header=None)
    date[0]=pd.to_datetime(date[0])
    date=date.set_index(0,drop=False)
    valid=date['2017':'2019.2.28'].iloc[:,:2].values
    d={'date':valid[:,0],'type':valid[:,1]}
    df=pd.DataFrame(data=d)
    df=df.sort_values(by='date')
    df=df.reset_index(drop=True)
    df=pd.get_dummies(df,columns=['type'])
    df.rename(columns={column:column[:5]+column[6:] for column in df.columns},inplace=True)
    return df

def build_features(raw_data):
    X_raw = copy.deepcopy(raw_data)
    X_raw['date']=pd.to_datetime(X_raw['date'],infer_datetime_format=True)
    X_raw['type'] = X_raw['type'].apply(lambda x:x[-1])
    X_raw['weekday'] = X_raw['date'].dt.weekday
    X_raw['is_month_start'] = X_raw['date'].dt.is_month_start
    X_raw['is_month_end'] = X_raw['date'].dt.is_month_end
    X_raw['ith_week'] = X_raw['date'].dt.week
    X_raw['month'] = X_raw['date'].dt.month
    lbl = preprocessing.LabelEncoder()
    X_raw['id'] = lbl.fit_transform(X_raw['id'].astype(str))#将提示的包含错误数据类型这一列进行转换
    X_raw['type'] = lbl.fit_transform(X_raw['type'].astype(str))
    return X_raw


In [ ]:
train_data = pd.read_csv('./data/da0030/train.csv', encoding='utf-8')
test_data = pd.read_csv('./data/da0030/predict.csv', encoding='utf-8')
others = getDateType()
train_more = build_features(train_data)
test_more = build_features(test_data)
train_plus=train_more.merge(others, on='date',how='left')
test_plus = test_more.merge(others, on ='date',how='left')